In [79]:
# Importer les bibliothèques nécessaires
import pandas as pd
import numpy as np
import statsmodels.api as sm
import statsmodels.formula.api as smf

In [80]:
import pandas as pd

dfs = {
    2019: pd.read_csv('final2019.csv'),
    2020: pd.read_csv('final2020.csv'),
    2021: pd.read_csv('final2021.csv'),
    2022: pd.read_csv('final2022.csv'),
    2023: pd.read_csv('final2023.csv'),
}

for year, df in dfs.items():
    print(f"--- Colonnes {year} ---")
    print(df.columns.tolist(), "\n")

/tmp/ipykernel_399563/1594908137.py:4: DtypeWarning: Columns (7,8,20,22,24,26,28,30,32) have mixed types. Specify dtype option on import or set low_memory=False.
  2019: pd.read_csv('final2019.csv'),
/tmp/ipykernel_399563/1594908137.py:5: DtypeWarning: Columns (7,8,20,22,28,30) have mixed types. Specify dtype option on import or set low_memory=False.
  2020: pd.read_csv('final2020.csv'),
/tmp/ipykernel_399563/1594908137.py:6: DtypeWarning: Columns (7,8,20,22,28,30,32) have mixed types. Specify dtype option on import or set low_memory=False.
  2021: pd.read_csv('final2021.csv'),
/tmp/ipykernel_399563/1594908137.py:7: DtypeWarning: Columns (7,8,20,22,24,28,30,32) have mixed types. Specify dtype option on import or set low_memory=False.
  2022: pd.read_csv('final2022.csv'),


--- Colonnes 2019 ---
['Date_établissement_DPE', 'Etiquette_DPE', 'Type_bâtiment', 'Année_construction', 'Période_construction', 'Surface_habitable_logement', 'Adresse_(BAN)', 'N°_département_(BAN)', 'Code_INSEE_(BAN)', 'Adresse_Normalisee', 'id_mutation', 'date_mutation', 'numero_disposition', 'nature_mutation', 'valeur_fonciere', 'adresse_numero', 'adresse_suffixe', 'adresse_nom_voie', 'adresse_code_voie', 'code_postal', 'code_commune', 'nom_commune', 'code_departement', 'ancien_code_commune', 'ancien_nom_commune', 'id_parcelle', 'ancien_id_parcelle', 'numero_volume', 'lot1_numero', 'lot1_surface_carrez', 'lot2_numero', 'lot2_surface_carrez', 'lot3_numero', 'lot3_surface_carrez', 'lot4_numero', 'lot4_surface_carrez', 'lot5_numero', 'lot5_surface_carrez', 'nombre_lots', 'code_type_local', 'type_local', 'surface_reelle_bati', 'nombre_pieces_principales', 'code_nature_culture', 'nature_culture', 'code_nature_culture_speciale', 'nature_culture_speciale', 'surface_terrain', 'longitude', '

/tmp/ipykernel_399563/1594908137.py:8: DtypeWarning: Columns (7,8,20,22,28,30) have mixed types. Specify dtype option on import or set low_memory=False.
  2023: pd.read_csv('final2023.csv'),


In [81]:
# 1) Concaténation
df = pd.concat(dfs.values(), ignore_index=True)
print("Shape après concaténation :", df.shape)

Shape après concaténation : (574251, 51)


In [87]:
# Juste après concaténation des fichiers, avant la conversion en datetime
print(df['date_mutation'].isna().sum())


0


In [83]:
# Refaire la conversion sans forcer le format jour/mois
df['date_mutation'] = pd.to_datetime(df['date_mutation'], errors='coerce')


In [86]:
import pandas as pd

# 1) Concaténation
df = pd.concat(dfs.values(), ignore_index=True)
print("Shape après concaténation :", df.shape)

# 2) Date → datetime + extraction de l'année
df['date_mutation'] = pd.to_datetime(df['date_mutation'], errors='coerce')
df['year'] = df['date_mutation'].dt.year

Shape après concaténation : (574251, 51)


In [88]:
# 1) Nettoyage de la variable DPE
df['dpe_clean'] = df['Etiquette_DPE'].str.upper().str.strip()

# 2) Création de l'indicatrice de traitement
# Le groupe de traitement est constitué des logements avec une étiquette DPE F ou G.
df['treat'] = df['dpe_clean'].isin(['F', 'G']).astype(int)

# 3) Définir l'indicateur 'post' qui vaut 1 si la date de mutation est postérieure à la date clé, sinon 0.
df['post_exact'] = (df['date_mutation'] >= '2021-08-24').astype(int)

# 4) Création de l'indicatrice DID qui est le produit de 'treat' et 'post_exact'
# Cette variable identifie les observations du groupe traité après la période de l'intervention.
df['did'] = df['treat'] * df['post_exact']

# 5) Création de l'indicatrice de groupe de contrôle (0 pour A, B, C, D, 1 pour F, G)
# Le groupe de contrôle est constitué des logements avec une étiquette DPE A, B, C ou D
df['control'] = df['dpe_clean'].isin(['A', 'B', 'C', 'D']).astype(int)



In [97]:
df[["surface_reelle_bati", "Surface_habitable_logement", "surface_terrain"]].isna().mean()


surface_reelle_bati           0.00000
Surface_habitable_logement    0.00000
surface_terrain               0.72123
dtype: float64

In [89]:
# 1) liste des contrôles
controls = [
    'surface_reelle_bati',  # Surface réelle bâtie
    'nombre_pieces_principales',  # Nombre de pièces principales
    'Année_construction',  # Année de construction
    'Type_bâtiment',  # Indicateur de valeurs manquantes dans l'année de construction
]

# 2) construction de la liste complète des variables du modèle
vars_model = ['valeur_fonciere', 'treat', 'post_exact', 'did', 'code_departement'] + controls

# 3) comptage des missing
missing = df[vars_model].isna().sum().rename('n_missing')
total   = df[vars_model].shape[0]
pct     = (missing / total * 100).round(2).rename('pct_missing')
print(pd.concat([missing, pct], axis=1))

# 4) création du jeu clean (drop NA)
df_mod = df.dropna(subset=vars_model)
print(f"\nObservations avant : {total:,}  –  après dropna : {len(df_mod):,}")


                           n_missing  pct_missing
valeur_fonciere                   29         0.01
treat                              0         0.00
post_exact                         0         0.00
did                                0         0.00
code_departement                   0         0.00
surface_reelle_bati                0         0.00
nombre_pieces_principales          0         0.00
Année_construction            105263        18.33
Type_bâtiment                      0         0.00

Observations avant : 574,251  –  après dropna : 468,960


In [90]:

# 1) Indicateur de missing + imputation pour l'année de construction
df['year_const_miss'] = df['Année_construction'].isna().astype(int)
median_year = df['Année_construction'].median()
df['Année_construction_imp'] = df['Année_construction'].fillna(median_year)

# 2) Nouvelle liste de contrôles (sans surface_terrain, avec l’imputation)
controls2 = [
    'surface_reelle_bati',
    'nombre_pieces_principales',
    'Type_bâtiment',
    'Année_construction_imp',
    'year_const_miss'
]

vars_model2 = ['valeur_fonciere', 'treat', 'post_exact', 'did', 'code_departement'] + controls2

# 3) Nouveau diagnostic des missing
missing2 = df[vars_model2].isna().sum().rename('n_missing')
pct2     = (missing2 / df.shape[0] * 100).round(2).rename('pct_missing')
print(pd.concat([missing2, pct2], axis=1))

# 4) Jeu « clean » après dropna
df_mod2 = df.dropna(subset=vars_model2)
df_mod2 = df_mod2[df_mod2['dpe_clean'].isin(['A', 'B', 'C', 'D', 'F', 'G'])]

print(f"\nObservations avant : {df.shape[0]:,}  –  après dropna : {len(df_mod2):,}")

                           n_missing  pct_missing
valeur_fonciere                   29         0.01
treat                              0         0.00
post_exact                         0         0.00
did                                0         0.00
code_departement                   0         0.00
surface_reelle_bati                0         0.00
nombre_pieces_principales          0         0.00
Type_bâtiment                      0         0.00
Année_construction_imp             0         0.00
year_const_miss                    0         0.00

Observations avant : 574,251  –  après dropna : 420,139


In [100]:

# Création du prix au mètre carré
df_mod2['code_departement'] = df_mod2['code_departement'].astype(str)
df_mod2["prix_m2"] = df_mod2["valeur_fonciere"] / df_mod2["surface_reelle_bati"]
df_mod2["log_prix_m2"] = np.log(df_mod2["prix_m2"])

# Variables D_AB (si DPE = A ou B) et D_FG (si DPE = F ou G)
df_mod2["D_AB"] = df_mod2["Etiquette_DPE"].isin(["A", "B"]).astype(int)
df_mod2["D_FG"] = df_mod2["Etiquette_DPE"].isin(["F", "G"]).astype(int)

In [103]:
# Si tu as une colonne de date, crée une variable "trimestre"
df_mod2['trimestre'] = df_mod2['date_mutation'].dt.to_period('Q')


In [104]:
print(df_mod2['trimestre'].unique())

<PeriodArray>
['2019Q4', '2019Q3', '2020Q4', '2020Q1', '2020Q2', '2020Q3', '2021Q4',
 '2021Q3', '2021Q1', '2021Q2', '2022Q4', '2022Q2', '2022Q1', '2022Q3',
 '2023Q4', '2023Q2', '2023Q3', '2023Q1']
Length: 18, dtype: period[Q-DEC]


1ère régression

In [ ]:
import statsmodels.formula.api as smf

df_mod2 = df_mod2.copy()
df_mod2['code_departement'] = df_mod2['code_departement'].astype(str)

# Formule du modèle Diff-in-Diff avec variables de contrôle
formula = (
    'np.log(prix_m2) ~ treat + post_exact + did + '
    '+ surface_reelle_bati + Type_bâtiment + '
    'Année_construction_imp + year_const_miss'
)

# Régression avec erreurs standards robustes clusterisées par département
model = smf.ols(formula, data=df_mod2).fit(
    cov_type='cluster',
    cov_kwds={'groups': df_mod2['code_departement']}
)

# Résultats
print(model.summary())



In [25]:
from statsmodels.stats.outliers_influence import variance_inflation_factor

# Sélection des variables numériques pour le VIF (sans les variables catégorielles comme code_type_local)
X_vif = df_mod2[[
    'treat',
    'post_exact',
    'did',
    'surface_reelle_bati',
    'Année_construction_imp',
    'year_const_miss'
]]

# Ajout de la constante (intercept)
X_vif = sm.add_constant(X_vif)

# Calcul du VIF
vif_data = pd.DataFrame()
vif_data['Variable'] = X_vif.columns
vif_data['VIF'] = [variance_inflation_factor(X_vif.values, i) for i in range(X_vif.shape[1])]

print(vif_data)


                 Variable        VIF
0                   const  95.080707
1                   treat   1.274063
2              post_exact   1.207479
3                     did   1.443576
4     surface_reelle_bati   1.011124
5  Année_construction_imp   1.004603
6         year_const_miss   1.029228


In [31]:
df_mod2['dpe_clean'].value_counts()



dpe_clean
D    201131
C    108333
F     61427
G     29991
B     13336
A      5921
Name: count, dtype: int64

2e régression : effet fixes

In [108]:
import statsmodels.formula.api as smf

# S'assurer que 'code_departement' est bien une chaîne
df_mod2['code_departement'] = df_mod2['code_departement'].astype(str)

# Formule avec effets fixes par département et année
formula = (
    'np.log(valeur_fonciere) ~ treat + did + '
    'C(code_departement) + C(year) + '
    'surface_reelle_bati + Type_bâtiment + '
    'Année_construction_imp + year_const_miss'
)

# Régression avec erreurs standards clusterisées par département
model_fe = smf.ols(formula=formula, data=df_mod2).fit(
    cov_type='cluster',
    cov_kwds={'groups': df_mod2['code_departement']}
)

# Résultats
print(model_fe.summary())



/opt/conda/lib/python3.12/site-packages/statsmodels/base/model.py:1894: ValueWarning: covariance of constraints does not have full rank. The number of constraints is 107, but rank is 11
  warnings.warn('covariance of constraints does not have full '


                               OLS Regression Results                              
Dep. Variable:     np.log(valeur_fonciere)   R-squared:                       0.380
Model:                                 OLS   Adj. R-squared:                  0.380
Method:                      Least Squares   F-statistic:                     2275.
Date:                     Sat, 19 Apr 2025   Prob (F-statistic):          7.80e-111
Time:                             14:15:14   Log-Likelihood:            -4.2725e+05
No. Observations:                   420139   AIC:                         8.547e+05
Df Residuals:                       420031   BIC:                         8.559e+05
Df Model:                              107                                         
Covariance Type:                   cluster                                         
                                coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------

3e régression : effets fixes plus fins

In [93]:
# Créer une nouvelle variable d'interaction département × année
df_mod2['dep_year'] = df_mod2['code_departement'].astype(str) + "_" + df_mod2['year'].astype(str)


In [109]:
import statsmodels.formula.api as smf

# Formule avec effets fixes croisés département × année
formula = (
    'np.log(valeur_fonciere) ~ treat + did + '
    'C(dep_year) + '
    'surface_reelle_bati + Type_bâtiment + '
    'Année_construction_imp + year_const_miss'
)

# Estimation avec erreurs standards clusterisées par département
model_cross_fe = smf.ols(formula=formula, data=df_mod2).fit(
    cov_type='cluster',
    cov_kwds={'groups': df_mod2['code_departement']}
)

print(model_cross_fe.summary())


/opt/conda/lib/python3.12/site-packages/statsmodels/base/model.py:1894: ValueWarning: covariance of constraints does not have full rank. The number of constraints is 470, but rank is 7
  warnings.warn('covariance of constraints does not have full '


                               OLS Regression Results                              
Dep. Variable:     np.log(valeur_fonciere)   R-squared:                       0.415
Model:                                 OLS   Adj. R-squared:                  0.414
Method:                      Least Squares   F-statistic:                     1633.
Date:                     Sat, 19 Apr 2025   Prob (F-statistic):           7.97e-97
Time:                             14:16:30   Log-Likelihood:            -4.1518e+05
No. Observations:                   420139   AIC:                         8.313e+05
Df Residuals:                       419668   BIC:                         8.365e+05
Df Model:                              470                                         
Covariance Type:                   cluster                                         
                                coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------